In [6]:
import numpy as np
import scipy as sc
import scipy.stats as st
import pandas as pd

import math
import matplotlib.pyplot as plt
import sys

import time

from StochasticProcess import *
from LongstaffSchwarz import *

## Longstaff Schwarz

Classical polynomial Basis

In [7]:
def basis_poly(x,k):
    return np.power(x,k)

##### Tests

In [8]:
#Payoff Call
def callPrice(x,k): return(max(x-k,0))

In [12]:
S = BS_model() #Standard BS (S0=100, sigma=.2, r=0, divs=0)
#Try to price American Call Option for a standard BS (S=K=100, r=0) with 5 basis vectors (d=5)
start = time.time()
print(LongstaffSchwarz(S, T=1, nbSteps=100, nbSimuls=1000, d=5, basis_projection=basis_poly, payoff=callPrice, arguments_payoff=(100,)))
print("Running time: ", time.time() - start)

7.586221907079174
Running time:  29.31965136528015


The price should be the price of the European call (see below).
Nevertheless, given the small number of simulations, there is still a lot of variance

In [13]:
#Compare to the price of european option (which should be the same)
S.computeCallPrice(100,1)

7.965567455405804

What if we add dividends ?

In [14]:
S = BS_model(divs=.1)
LongstaffSchwarz(S, T=1, nbSteps=100, nbSimuls=1000, d=5, basis_projection=basis_poly, payoff=callPrice, arguments_payoff=(100,))

4.92646299720192

In [15]:
S.computeCallPrice(100,1)

3.753418388256833

American call becomes more expensive --> makes sense

What about a put option ?

In [17]:
#Put option payoff
def putPrice(x,k): return(max(k-x,0))

In [18]:
LongstaffSchwarz(BS_model(), T=1, nbSteps=100, nbSimuls=1000, d=5, basis_projection=basis_poly, payoff=putPrice, arguments_payoff=(100,))

6.9347530095631456

In [19]:
BS_model().computePutPrice(K=100, T=1)

7.965567455405804

We still have an under evaluation by the LS algorithm

With dividends:

In [21]:
LongstaffSchwarz(BS_model(divs=.1), T=1, nbSteps=100, nbSimuls=1000, d=5, basis_projection=basis_poly, payoff=putPrice, arguments_payoff=(100,))

11.493735198762733

In [22]:
BS_model(divs=.1).computePutPrice(K=100, T=1)

13.269676584660886

It looks like the projection on the canonique polynomial basis performs poorly and under-evaluate the price too much (the price is often less than the price of the european corresponding security)

Let's evaluate the influence of the size of the basis in the case of the canonique polynomial basis, and the influence of the number of iterations, for N = 100 time steps

In [666]:

#!!!!!!!!!!!Warning!!!!!!!!!!: This might (really) take a while - check my results above for an estimation of the running time


D = np.arange(1, 6, 1)
M = np.array([10, 100, 1000, 10000])
means = np.zeros((len(D), len(M)))
stdev = np.zeros((len(D), len(M)))
times_means = np.zeros((len(D), len(M)))

model = BS_model()


start_total_time = time.time()

for i in range(len(D)):
    for j in range(len(M)):
        
        simuls = np.zeros(10)
        for k in range(10):
            start_time = time.time()
            simuls[k] = LongstaffSchwarz(model, T=1, nbSteps=100, nbSimuls=M[j], d=D[i], basis_projection=basis_poly, payoff=callPrice, arguments_payoff=(100,))
            times_means[i,j] += 0.1*(time.time() - start_time)
            
        #Time of these operations is negligible 
        means[i,j] = np.average(simuls)
        stdev[i,j] = np.sqrt(np.var(simuls, ddof=1))
            
total_time=time.time() - start_total_time

true_result = model.computeCallPrice(K=100,T=1)

In [669]:
print("Running time (minutes): ",total_time/60.)

Running time (minutes):  24.599308915932973


In [676]:
#Average running times (in seconds)
pd.DataFrame(times_means, index=D, columns=M)

,10,100,1000,10000
1,0.025432,0.232977,1.606804,15.908964
2,0.033021,0.199566,1.890849,22.022514
3,0.048176,0.253222,2.367267,27.207706
4,0.062333,0.307578,2.925778,32.146507
5,0.078191,0.373505,3.378463,36.526902


In [673]:
#Means
pd.DataFrame(means, index=D, columns=M)

,10,100,1000,10000
1,5.343280,5.481413,5.104498,5.057530
2,10.460480,7.687859,7.026671,6.998446
3,11.712747,8.247258,7.814078,7.520951
4,8.613475,9.040933,7.731983,7.486816
5,11.029250,8.777395,7.486101,7.472610


In [678]:
print("Correct price: ", true_result)

Correct price:  7.808358491192365


In [677]:
#Stdevs
pd.DataFrame(stdev, index=D, columns=M)

,10,100,1000,10000
1,1.657835,0.896406,0.175822,0.090268
2,4.834249,0.945276,0.218042,0.089543
3,3.958621,1.044746,0.480108,0.105985
4,2.580378,0.595337,0.281235,0.091064
5,2.704211,1.355896,0.262980,0.119394


##### Influence of the polynomial basis

In [23]:
def Laguerre_polyn(x, k):
    if(k<0):
        sys.exit("k should be a positive integer")
    if(k==0):
        return 1
    elif (k==1):
        return 1-x
    else:
        n = k-1
        return ( -((x-2*n - 1)*Laguerre_polyn(x,k-1) + n*Laguerre_polyn(x,k-2))/float(n+1) )
    
def Laguerre_polyn_exp(x,k):
    return math.exp(-x/2)*Laguerre_polyn(x,k)

In [48]:
start = time.time()
print(LongstaffSchwarz(BS_model(), T=1, nbSteps=100, nbSimuls=10000, d=4, basis_projection=Laguerre_polyn, payoff=callPrice, arguments_payoff=(100,)))
print("Running Time: ", time.time()-start)

7.531914079256549
Running Time:  114.3780403137207


In [40]:
print("True price= ", BS_model().computeCallPrice(100,1))

True price=  7.808358491192365


In [41]:
start = time.time()
print(LS.LongstaffSchwarz(SP.BS_model(), T=1, nbSteps=100, nbSimuls=10000, d=4, basis_projection=basis_poly, payoff=callPrice, arguments_payoff=(100,)))
print("Running Time: ", time.time()-start)

7.605212204364818
Running Time:  118.1510899066925


### Check Longstaff & Schwarz results (2001)

In [43]:
model1 = BS_model(S0=36, vol=.2, r=.06)
model1.computePutPrice(40,1.)

3.84430779159684

In [70]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=50, nbSimuls=10000, d=3, basis_projection=Laguerre_polyn, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.4623205003199455
Running Time:  22.443670988082886


In [71]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=50, nbSimuls=10000, d=3, basis_projection=Laguerre_polyn_exp, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.133054961792035
Running Time:  31.87619185447693


Classical Laguerre Polynomials gives a better regression estimate

In [73]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=50, nbSimuls=100000, d=3, basis_projection=Laguerre_polyn, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.413709155947841
Running Time:  211.6997721195221


Convergence was enough with 10000 simulations. What if we add some time steps ?

In [74]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=100, nbSimuls=10000, d=3, basis_projection=Laguerre_polyn, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.384821976164643
Running Time:  44.06898260116577


No significant difference. With 2 basis functions?

In [75]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=50, nbSimuls=10000, d=2, basis_projection=Laguerre_polyn, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.295047980674473
Running Time:  9.808556318283081


The result diverges - underfitting

With classical basis:

In [76]:
start = time.time()
print(LongstaffSchwarz(model1, T=1, nbSteps=50, nbSimuls=10000, d=3, basis_projection=basis_poly, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.393381331258802
Running Time:  36.31278967857361


Laguerre polyn give a better estimate

In [78]:
start = time.time()
print(LongstaffSchwarz(model1, T=2, nbSteps=100, nbSimuls=10000, d=3, basis_projection=Laguerre_polyn, payoff=putPrice, arguments_payoff=(40,)))
print("Running Time: ", time.time()-start)

4.668705957802519
Running Time:  46.784849643707275
